The following notebook shows insights about driver's dataset for a mobility as a service company (available here: https://data.world/).

The scope is to use SQL to provide insights about drivers behaviour and other key metrics such as retention and conversion rate.

In the first part, we are going to import the datasets and perform some data manipulation. This is needed to avoid further complications, especially when dealing with dates, as jupyter notebook is based on SQL lite.

In the second part, we are going to focus on the insights.

### 1. Importing, renaming variables and converting dates

In [10]:
import pandasql as pdsql
import pandas as pd

In [11]:
# Run this cell to define a SQL function
def run_sql(sql_string):
    try:
        result = pdsql.sqldf(sql_string)
        return result
    except Exception as e:
        print('''Query Excecution Failed''')
        print('Error: ',e)

def to_csv(table, target_file_name):
    table.to_csv(target_file_name+'.csv')

In [ ]:
# It is possible to export the data like this
data_to_export = run_sql('''select * from event limit 10''')

# Export to a csv file:
to_csv(data_to_export, 'data_export_demo')

# After running this cell, you'll find a new data_export_demo.CSV file in the folder

In [86]:
# Load table

file_path=r'C:\Users\Marco\Desktop'
file_driver='/driver1.csv'
file_driver_activity='/driver_activity1.csv'

driver = pd.read_csv(file_path+file_driver, encoding = "ISO-8859-1")
driver.head(5)

,ï»¿id_driver,date_registration,driver_rating,gold_level_count,receive_marketing,country_code,service_type
0,15260831,2015-11-18,4.85,2.0,True,ES,TAXI
1,58194598,2020-03-12,4.79,26.0,True,DE,PHV
2,21139412,2017-05-15,4.76,10.0,True,DE,TAXI
3,48382094,2019-03-20,4.91,3.0,False,DE,TAXI
4,52914047,2019-08-02,4.96,10.0,True,DE,PHV


In [93]:
driver_activity = pd.read_csv(file_path+file_driver_activity, encoding = "ISO-8859-1")
driver_activity.head(5)

,ï»¿id_driver,active_date,offers,bookings,bookings_cancelled_by_passenger,bookings_cancelled_by_driver,rides
0,3,2020-01-01,65.0,18.0,3.0,4.0,11.0
1,3,2020-01-02,3.0,2.0,0.0,1.0,1.0
2,3,2020-01-03,1.0,1.0,0.0,0.0,1.0
3,3,2020-01-04,3.0,4.0,0.0,0.0,4.0
4,3,2020-01-05,6.0,4.0,0.0,1.0,3.0


In [89]:
#renaming driver_id in both dataset
driver=driver.rename(columns = { "ï»¿id_driver" : "id_driver"})
driver.head(5)

,id_driver,date_registration,driver_rating,gold_level_count,receive_marketing,country_code,service_type
0,15260831,2015-11-18,4.85,2.0,True,ES,TAXI
1,58194598,2020-03-12,4.79,26.0,True,DE,PHV
2,21139412,2017-05-15,4.76,10.0,True,DE,TAXI
3,48382094,2019-03-20,4.91,3.0,False,DE,TAXI
4,52914047,2019-08-02,4.96,10.0,True,DE,PHV


In [94]:
driver_activity=driver_activity.rename(columns = { "ï»¿id_driver" : "id_driver"})
driver_activity.head(5)

,id_driver,active_date,offers,bookings,bookings_cancelled_by_passenger,bookings_cancelled_by_driver,rides
0,3,2020-01-01,65.0,18.0,3.0,4.0,11.0
1,3,2020-01-02,3.0,2.0,0.0,1.0,1.0
2,3,2020-01-03,1.0,1.0,0.0,0.0,1.0
3,3,2020-01-04,3.0,4.0,0.0,0.0,4.0
4,3,2020-01-05,6.0,4.0,0.0,1.0,3.0


In SQL lite (version used in jupyter) date functions only work if it the dates are in the format YYYY-MM-DD HH:MM:SS.SSS.

We need to do some work around to convert active_date and date_registration.

In [90]:
driver = run_sql('''
SELECT  *
        , (date_registration || ' ' || "00" || ':' || "00" || ':' || "00") as date_registration1        
FROM driver
''')

driver = driver.drop(['date_registration'], axis=1) 
driver=driver.rename(columns = { "date_registration1" : "date_registration"})
driver.head(5)

,id_driver,driver_rating,gold_level_count,receive_marketing,country_code,service_type,date_registration
0,15260831,4.85,2.0,1,ES,TAXI,2015-11-18 00:00:00
1,58194598,4.79,26.0,1,DE,PHV,2020-03-12 00:00:00
2,21139412,4.76,10.0,1,DE,TAXI,2017-05-15 00:00:00
3,48382094,4.91,3.0,0,DE,TAXI,2019-03-20 00:00:00
4,52914047,4.96,10.0,1,DE,PHV,2019-08-02 00:00:00


In [95]:
driver_activity = run_sql('''
SELECT  *
        , (active_date || ' ' || "00" || ':' || "00" || ':' || "00") as active_date1        
FROM driver_activity
''')

driver_activity = driver_activity.drop(['active_date'], axis=1) 
driver_activity=driver_activity.rename(columns = { "active_date1" : "active_date"})
driver_activity.head(5)

,id_driver,offers,bookings,bookings_cancelled_by_passenger,bookings_cancelled_by_driver,rides,active_date
0,3,65.0,18.0,3.0,4.0,11.0,2020-01-01 00:00:00
1,3,3.0,2.0,0.0,1.0,1.0,2020-01-02 00:00:00
2,3,1.0,1.0,0.0,0.0,1.0,2020-01-03 00:00:00
3,3,3.0,4.0,0.0,0.0,4.0,2020-01-04 00:00:00
4,3,6.0,4.0,0.0,1.0,3.0,2020-01-05 00:00:00


We finished with the data manipulation part, we can start looking at the insights.

## 2. Insights

We will mainly focus on the rides, because this is what brings the revenue

- First of all, we can see that the rides are split 60% from Germany and 40% from Spain

In [83]:
#Let's see how the rides are split per country 
run_sql('''
        select distinct country_code
        , sum(rides) OVER (PARTITION BY country_code) as rides_per_country
        , sum(rides) OVER () as rides_total
        , (sum(rides) OVER (PARTITION BY country_code))/(sum(rides) OVER ()) as rides_country_share
        from driver d
        left join driver_activity da on d.id_driver = da.id_driver
        order by rides_country_share desc
        ''')

,country_code,rides_per_country,rides_total,rides_country_share
0,DE,1822356.0,3022966.0,0.602837
1,ES,1200610.0,3022966.0,0.397163


- Most of the rides come from a taxi

In [52]:
#rides split per service type 
run_sql('''
        select distinct service_type
        , sum(rides) OVER (PARTITION BY service_type) as rides_per_service
        , sum(rides) OVER () as rides_total
        , (sum(rides) OVER (PARTITION BY service_type))/(sum(rides) OVER ()) as rides_service_share
        from driver d
        left join driver_activity da on d.id_driver = da.id_driver
        order by service_type desc
        ''')

,service_type,rides_per_service,rides_total,rides_service_share
0,TAXI,"2,990,387.00","3,022,966.00",0.99
1,PHV,"32,579.00","3,022,966.00",0.01


We can see who are the top 10 drivers with higher ride_conversion_rate, considering drivers that had at least 5 rides.

It is surprising to see that some drivers have a rate higher than 100%.

I assume there is also the possibility to accept rides that are not booked from the app, but just from people who stopped the driver on the street. 

In [84]:
run_sql('''
        select distinct d.id_driver
        , d.country_code
        , d.driver_rating
        , sum(da.offers) as tot_offers
        , sum(da.bookings) as tot_bookings
        , sum(da.rides) as tot_rides
        , sum(da.rides)/sum(da.offers) as ride_conversion_rate
        from driver d
        left join driver_activity da on d.id_driver = da.id_driver
        group by d.id_driver
        having sum(da.rides) > 5
        order by ride_conversion_rate desc        
        limit 10
        ''')

,id_driver,country_code,driver_rating,tot_offers,tot_bookings,tot_rides,ride_conversion_rate
0,19906130,ES,5.00,104.0,158.0,140.0,1.346154
1,14038077,DE,4.90,45.0,79.0,59.0,1.311111
2,17312397,ES,4.86,44.0,64.0,54.0,1.227273
3,16282198,DE,5.00,5.0,6.0,6.0,1.200000
4,6461217,DE,4.95,35.0,49.0,39.0,1.114286
5,19906179,ES,4.99,111.0,131.0,113.0,1.018018
6,18903390,DE,4.80,6.0,6.0,6.0,1.000000
7,18047235,DE,4.89,6.0,6.0,6.0,1.000000
8,17175552,DE,4.95,6.0,6.0,6.0,1.000000
9,15278227,DE,4.82,9.0,9.0,9.0,1.000000


We can create 3 different groups based on the ride conversion rate (high, medium and low) and see how are the drivers distributed in those categories.

According to the threshold we decided to use, 71% of the drivers fall in the "Low" category.

In [96]:
ride_conversion_rate = run_sql('''
            select distinct ride_conversion_category
                    , count(id_driver) OVER (PARTITION BY ride_conversion_category) as n_user_category
                    , count(id_driver) OVER () as n_user_tot
            from (
             select distinct d.id_driver
                    , case when sum(da.rides)/sum(da.offers) < 0.3 
                      or sum(da.rides)/sum(da.offers) is null then "Low"
                      when sum(da.rides)/sum(da.offers) between 0.3 and 0.75 then "Medium"
                      else "High" 
                      end as ride_conversion_category        
                    from driver d
                    left join driver_activity da on d.id_driver = da.id_driver
                    group by d.id_driver
                )

        ''')

ride_conversion_rate["n_user_tot"] = ride_conversion_rate["n_user_tot"].astype(float)
ride_conversion_rate["n_user_tot"] = ride_conversion_rate["n_user_tot"].astype(float)

run_sql('''
select distinct ride_conversion_category
                    , n_user_category
                    , n_user_tot
                    , n_user_category/n_user_tot as category_share
            from ride_conversion_rate
            order by category_share desc
''')

,ride_conversion_category,n_user_category,n_user_tot,category_share
0,Low,26270,36771.0,0.714422
1,Medium,10109,36771.0,0.274918
2,High,392,36771.0,0.010661


We can have a look at the funnel analysis to understand:
    
- The proportion of offers converted to bookings
- The proportion of bookings converted to rides
- The proportion of offers converted to rides

We can see that the the number of offers converted to booking is quite small (27%).

However, the number of booking converted to rides it is quite high (84%).

It seems that the drivers are declining many offers (73% of them). However the customers, once the offer is accepted, are likely to not to cancel their booking (only 16% canceled it).

We might consider that the offers showed to the drivers are not the optimal ones, maybe they are too far or the time is not suitable for them. We could help the drivers to understand which are the areas and times with the highest demand.

If we could improve the conversion from offers to bookings, we will be also able to increase the total conversion from offers to rides (23%).

In [141]:
run_sql('''
        SELECT sum(offers) as tot_offers
              , sum(bookings) as tot_bookings
              , sum(rides) as tot_rides
              , sum(bookings)/sum(offers) as offer_to_booking_conv_rate
              , sum(rides)/sum(bookings) as booking_to_ride_conv_rate
              , sum(rides)/sum(offers) as offers_to__ride_conv_rate
        FROM driver_activity da     
        ''')

,tot_offers,tot_bookings,tot_rides,offer_to_booking_conv_rate,booking_to_ride_conv_rate,offers_to__ride_conv_rate
0,13037445.0,3567970.0,3003708.0,0.273671,0.841853,0.230391


The driver_activity table starts from JAN 2020, but there are no drivers that are in both tables (driver and driver_activity) with date_registration = 2020. As shown in the code below, the INNER JOIN does not show any result.

So, the minimum retention day is 680 days (ca 2 years and 10 months). As shown in the code below.

That is why, I am going to build the the retention table starting from 3 years on.

The retention table does not really help to get insights, as the data are structured in a way that is difficult to see the retention in the short term period. Of course, after more than 3 years the number of drivers still using the app is quite small. 

It would have been interesting to see how was the drivers behaviour in the short term period. E.g. checking after one week, one month, 6 months and max one year after the date_registration how many of them were actually still providing the service.

In [112]:
run_sql('''
            SELECT d.id_driver
            FROM (
                    SELECT id_driver 
                            , date_registration
                    FROM driver 
                    WHERE strftime('%Y', date_registration) = '2020'
                 ) d
            INNER JOIN driver_activity da ON d.id_driver = da.id_driver
''')

,id_driver


In [110]:
run_sql('''
SELECT min(retention_day) as min_retention_day
FROM (
SELECT DISTINCT d.id_driver
                , julianday(da.active_date) - julianday(d.date_registration) as 'retention_day'
                , da.active_date
                , d.date_registration
FROM driver d
LEFT JOIN driver_activity da on d.id_driver = da.id_driver 
    )
        ''')

,min_retention_day
0,680.0


In [137]:
run_sql('''
                SELECT date_registration
                        , sum(case when retention_day=1095 then 1 else 0 END) as '3_years'
                        , sum(case when retention_day=1102 then 1 else 0 END) as '3_years_&_1_week'
                        , sum(case when retention_day=1109 then 1 else 0 END) as '3_years_&_2_weeks'   
                        , sum(case when retention_day=1125 then 1 else 0 END) as '3_years_&_1_month'
                FROM (
                SELECT DISTINCT d.id_driver
                        , julianday(da.active_date) - julianday(d.date_registration) as 'retention_day'
                        , date(d.date_registration) as date_registration
                FROM driver d
                LEFT JOIN driver_activity da on d.id_driver = da.id_driver
                WHERE retention_day in (1095, 1102, 1109, 1125)
                    )   
                GROUP BY date_registration
        ''')

,date_registration,3_years,3_years_&_1_week,3_years_&_2_weeks,3_years_&_1_month
0,2016-12-02,0,0,0,4
1,2016-12-05,0,0,0,4
2,2016-12-06,0,0,0,2
3,2016-12-07,0,0,0,4
4,2016-12-08,0,0,0,8
...,...,...,...,...,...
141,2017-06-26,7,0,0,0
142,2017-06-27,8,0,0,0
143,2017-06-28,5,0,0,0
144,2017-06-29,4,0,0,0
